In [1]:
# Import needed packages
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model
import torch
import torchvision.transforms as T
from torchvision.models.detection import maskrcnn_resnet50_fpn
import shutil
from PIL import Image

In [2]:
# This function is to process image that need to be learn by the model in order to hit a higher accuracy for the model
def process_img(image):
    # Preprocess image into a PyTorch tensor
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    
    # Load the pre-trained model Mask R-CNN model with a ResNet-50 backbone and FPN(Feature Pyramid Network)
    model = maskrcnn_resnet50_fpn(pretrained=True)
    # Evaluate the model
    model.eval()
    
    # Disable the gradient computation and use the model to predict the input image
    with torch.no_grad():
        predictions = model([input_image])
    
    # Extract the masks and boxes from the prediction and return them as a numpy array
    masks = predictions[0]['masks'].cpu().numpy()
    boxes = predictions[0]['boxes'].cpu().numpy()

    # Calculate the areas of the boxes
    areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    
    # Find the index of largest value in the areas and use boxes to select the largest area using the index found
    main_object_index = np.argmax(areas)
    main_object_box = boxes[main_object_index]

    # Convert the box coordinates to integers
    main_object_box = main_object_box.astype(int)

    # Extract the coordinates of the box
    x, y, w, h = main_object_box

    # Create a binary mask for the main object
    main_object_mask = masks[main_object_index, 0] > 0.5

    # Apply the binary mask to the input image
    result = cv2.bitwise_and(image, image, mask=main_object_mask.astype(np.uint8))

    # Create a background with transparent 
    transparent_background = np.zeros_like(image)

    # Replace the object background as transparent 
    result[np.where(main_object_mask == 0)] = transparent_background[np.where(main_object_mask == 0)]

    # Convert the result from BGR into BGRA format to make sure that the background is transparent
    result_with_transparency = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result_with_transparency[np.where((result == [0, 0, 0]).all(axis=2))] = [0, 0, 0, 0]

    # Find the contours of the object
    contours, _ = cv2.findContours(main_object_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the minimum area rectangle around the contours and convert to integer
    rect = cv2.minAreaRect(contours[0])
    box = cv2.boxPoints(rect).astype(np.int32)

    # Crop the image using the box
    x, y, w, h = cv2.boundingRect(box)
    cropped_image = result_with_transparency[y:y+h, x:x+w]
    
    # Return a cropped image
    return cropped_image

In [3]:
# This function is to take an amount of frame from a video
def take_frame(video):
    # Set the output directory and create a new directory in the os
    output_directory = f'{video}_img'
    os.mkdir(output_directory)
    
    # Open the video and compute how many FPS(Frame Per Second) of video being process
    open_video = cv2.VideoCapture(video)
    frame = int(open_video.get(cv2.CAP_PROP_FPS))
    
    # Set the variable frame_count and x to zero and one
    frame_count = 0
    x = 1
    # While the video is still open, read the next frame of the video
    while open_video.isOpened():
        left, frame_data = open_video.read()

        # Check if the frame was successfully read, if the frame is unsuccessfully read, break it
        if not left:
            break
        # Else if the frame_count divide by frame wouldn't have a remainder
        elif frame_count % frame == 0:
            # Process the image in order to take the main object and crop it and back the background be transparent
            img = process_img(frame_data)
            # If the image is available
            if img is not None and img.size != 0:
                # Save it to the directory
                cv2.imwrite(output_directory + '/' + f'frame{x}.png', img)
            # Add one to the variable x
            x += 1
        
        # Add one to the variable frame_count
        frame_count += 1
    
    # Realease the video
    open_video.release()

In [ ]:
# This few example for loop is to show where we take the video of the car and take the frame of the car from the video 
# in order to collect the car image data and let us to train the model to differentiate the car type

for a in range(18):
    take_frame(f'Training Video/SEDAN/SAGA{a+1}.mov')
    
for b in range(19):
    take_frame(f'Training Video/HATCHBACK/IRIZ{b+1}.mov')
    
for c in range(32):
    take_frame(f'Training Video/SUV/X50{c+1}.mov')
    
for d in range(23):
    take_frame(f'Training Video/SUV/X70{d+1}.mov')

In [4]:
# This function is to combine all the image of one folder which contain an amount of folder into a folder only 
def combine_file(source, destination):
    # Set the variable x as one
    x = 1
    # If the path not exist
    if not os.path.exists(destination):
        # Make directory for the destination
        os.makedirs(destination)
    
    # Loop all the folder in the directory
    for folder in os.listdir(source):
        # Join the filename with the directory
        source_file = os.path.join(source, folder)
        # Loop all the file in the folder
        for file in os.listdir(source_file):
            # Join the filename with the folder name
            source_path = os.path.join(source_file, file)
            # If the file exist
            if os.path.isfile(source_path):
                # Set the destination path to the destination follow by the x variable
                destination_path = os.path.join(destination, str(x))
                # Copy from the directory to the destination
                shutil.copy(source_path, destination_path)
                # Add one to the variable x
                x += 1
                # Rename the new file
                new_file = os.path.splitext(destination_path)[0] + '.png'
                os.rename(destination_path, new_file)

In [5]:
# This function is to rotate the image of the folder with certain degree
def rotate_images(folder_path, degrees):
    # Loop all the file in the folder
    for filename in os.listdir(folder_path):
        # If the filename is end with .jpg and .png
        if filename.endswith(".jpg") or filename.endswith(".png"): 
            # Set the variable image_path to the join of folder name and filename
            image_path = os.path.join(folder_path, filename)
            try:
                # Open the image file
                with Image.open(image_path) as image:
                    # Rotate the image and save it 
                    rotated_image = image.rotate(degrees, expand=True)
                    rotated_image.save(image_path)
                    print(f"Rotated {filename} successfully.")
            # If fail to open, print Failed to rotate in the console
            except Exception as e:
                print(f"Failed to rotate {filename}: {str(e)}")

In [ ]:
# This is example of combining file and rotate the image
source = 'Training Data/HATCHBACK'
destination = 'Training Data/SUV'
combine_file(source, destination)
rotate_images('Template Photo', 180)

In [6]:
# Define the path to the directory which contain the car images which need to use for training
data_dir = 'Training Data'

# Define the car models in an array
car_models = ['HATCHBACK', 'SEDAN', 'SUV']

# Initialize empty lists to store the image data and labels
images = []
labels = []

# Define the input image dimensions
input_shape = (128, 128, 1)  # Assuming color images

# Loop every item in the car_models array
for car_model in car_models:
    # Join the directory with the car_model item to be the new path
    part_dir = os.path.join(data_dir, car_model)
    # Loop the file in the car model item
    for image_file in os.listdir(part_dir):
        # Set the image_path by joining the file directory with the name of the image file
        image_path = os.path.join(part_dir, image_file)
        # Read the image
        image = cv2.imread(image_path)
        # If the image is available
        if image is not None:
            # Resize the image to a fixed size
            image = cv2.resize(image, input_shape[:2])
            # Convert the image from RGB to Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            # Append the image into the image array and append the car model into the labels array
            images.append(image)
            labels.append(car_model)
            
# Convert the image and label lists to numpy arrays
X = np.array(images)
y = np.array(labels)

# Create a label map dictionary that map each label in the car_models list to the corresponding index
num_classes = len(car_models)
label_map = {label: index for index, label in enumerate(car_models)}

# Convert array y of labels to an array corresponding to indices based on the provided label_map
y = np.array([label_map[label] for label in y])

# Shuffle the data
shuffle_indices = np.arange(X.shape[0])
np.random.shuffle(shuffle_indices)
X = X[shuffle_indices]
y = y[shuffle_indices]

# Split the data into 90 percent training and 10 percent testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the input data to match the input shape of the model
X_train = np.reshape(X_train, (-1,) + input_shape)
X_test = np.reshape(X_test, (-1,) + input_shape)

# Create a CNN model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model with adam optimizer and loss function is using sparse_categorical_crossentropy
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the loss and accuracy of the model
print(f'Loss : {loss}, Accuracy : {accuracy}')

Epoch 1/10
60/60 [==============================] - 30s 495ms/step - loss: 1.0492 - accuracy: 0.5042 - val_loss: 0.7868 - val_accuracy: 0.6635
Epoch 2/10
60/60 [==============================] - 30s 492ms/step - loss: 0.7253 - accuracy: 0.6928 - val_loss: 0.6041 - val_accuracy: 0.7488
Epoch 3/10
60/60 [==============================] - 29s 491ms/step - loss: 0.4478 - accuracy: 0.8288 - val_loss: 0.4273 - val_accuracy: 0.8294
Epoch 4/10
60/60 [==============================] - 29s 484ms/step - loss: 0.3000 - accuracy: 0.8867 - val_loss: 0.3984 - val_accuracy: 0.8531
Epoch 5/10
60/60 [==============================] - 28s 474ms/step - loss: 0.2444 - accuracy: 0.9078 - val_loss: 0.3019 - val_accuracy: 0.8720
Epoch 6/10
60/60 [==============================] - 28s 463ms/step - loss: 0.1272 - accuracy: 0.9557 - val_loss: 0.3610 - val_accuracy: 0.8483
Epoch 7/10
60/60 [==============================] - 28s 462ms/step - loss: 0.0599 - accuracy: 0.9821 - val_loss: 0.2904 - val_accuracy: 0.9194

In [7]:
# This function is same as the above function name process_img.
# Both of the function doing the same thing but with different number of input parameter.
def process_img_path(image, a):
    # Read the given image using cv2
    image = cv2.imread(image)
    # Preprocess image into a PyTorch tensor
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    
    # Load the pre-trained model Mask R-CNN model with a ResNet-50 backbone and FPN(Feature Pyramid Network)
    model = maskrcnn_resnet50_fpn(pretrained=True)
    # Evaluate the model
    model.eval()

    # Disable the gradient computation and use the model to predict the input image
    with torch.no_grad():
        predictions = model([input_image])
        
    # Extract the masks and boxes from the prediction and return them as a numpy array
    masks = predictions[0]['masks'].cpu().numpy()
    boxes = predictions[0]['boxes'].cpu().numpy()

    # Calculate the areas of the boxes
    areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])

    # Find the index of largest value in the areas and use boxes to select the largest area using the index found
    main_object_index = np.argmax(areas)
    main_object_box = boxes[main_object_index]

    # Convert the box coordinates to integers
    main_object_box = main_object_box.astype(int)

    # Extract the coordinates of the box
    x, y, w, h = main_object_box

    # Create a binary mask for the main object
    main_object_mask = masks[main_object_index, 0] > 0.5

    # Apply the binary mask to the input image
    result = cv2.bitwise_and(image, image, mask=main_object_mask.astype(np.uint8))

    # Create a background with transparent 
    transparent_background = np.zeros_like(image)

    # Replace the object background as transparent 
    result[np.where(main_object_mask == 0)] = transparent_background[np.where(main_object_mask == 0)]

    # Convert the result from BGR into BGRA format to make sure that the background is transparent
    result_with_transparency = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result_with_transparency[np.where((result == [0, 0, 0]).all(axis=2))] = [0, 0, 0, 0]

    # Find the contours of the object
    contours, _ = cv2.findContours(main_object_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the minimum area rectangle around the contours and convert to integer
    rect = cv2.minAreaRect(contours[0])
    box = cv2.boxPoints(rect).astype(np.int32)

    # Crop the image using the box
    x, y, w, h = cv2.boundingRect(box)
    cropped_image = result_with_transparency[y:y+h, x:x+w]
    
    # If the image is available, save it 
    if cropped_image is not None and cropped_image.size > 0:
       cv2.imwrite(f'ans{a}.png', cropped_image)
    # Or else use a black image to replce it and save it
    else:
       cv2.imwrite(f'ans{a}.png', cv2.imread('Error Image.jpg'))

In [8]:
# This function is to detect whether the image contains car to prevent wrong prediction 
def image_detect(image_path, x):
    # The correct_num variable is set to zero and it is use to count the number of image detected car
    correct_num = 0
    # Load and read the template image of the car using cv2
    template1 = cv2.imread('Template Photo/template1.png', cv2.IMREAD_GRAYSCALE)
    template2 = cv2.imread('Template Photo/template2.png', cv2.IMREAD_GRAYSCALE)
    template3 = cv2.imread('Template Photo/template3.png', cv2.IMREAD_GRAYSCALE)
    template4 = cv2.imread('Template Photo/template4.png', cv2.IMREAD_GRAYSCALE)
    template5 = cv2.imread('Template Photo/template5.png', cv2.IMREAD_GRAYSCALE)
    template6 = cv2.imread('Template Photo/template6.png', cv2.IMREAD_GRAYSCALE)
    template7 = cv2.imread('Template Photo/template7.png', cv2.IMREAD_GRAYSCALE)
    template8 = cv2.imread('Template Photo/template8.png', cv2.IMREAD_GRAYSCALE)
    template9 = cv2.imread('Template Photo/template9.png', cv2.IMREAD_GRAYSCALE)
    template10 = cv2.imread('Template Photo/template10.png', cv2.IMREAD_GRAYSCALE)
    template11 = cv2.imread('Template Photo/template11.png', cv2.IMREAD_GRAYSCALE)
    template12 = cv2.imread('Template Photo/template12.png', cv2.IMREAD_GRAYSCALE)
    # Set up an array to put in all the template of the car
    template = [template1, template2, template3, template4, template5, template6, template7, template8, template9, template10, template11, template12]
    
    # Read and process the input image and save it
    input_image = process_img_path(image_path, x)
    input_image = cv2.imread(f"ans{x}.png", cv2.IMREAD_GRAYSCALE)
    
    # Loop all the template 
    for x in range(12):
        # Calculate the ratio of width and height of the template image compare to the input image
        ratio_width =  template[x].shape[1] / input_image.shape[1]
        ratio_height = template[x].shape[0] / input_image.shape[0]
        # Convert the template image into the similar size of the input image
        if ratio_width > ratio_height:
            template[x] = cv2.resize(template[x], (int(template[x].shape[1]/(ratio_width+0.01)), int(template[x].shape[0]/(ratio_width+0.01))))
        elif ratio_width < ratio_height:
            template[x] = cv2.resize(template[x], (int(template[x].shape[1]/(ratio_height+0.01)), int(template[x].shape[0]/(ratio_height+0.01))))
    
    # Loop all the input image with the template image
    for i in range(12):
        # If the input image shape is larger than 200x200
        if input_image.shape[1] > 200 and input_image.shape[0] > 200:
            # Use cv2 to match the template image and input image and put the result into the result variable
            result = cv2.matchTemplate(input_image, template[i], cv2.TM_CCOEFF_NORMED)
            # Set the threshold to 0.4
            threshold = 0.4  
            # Find indices of element which is greater than threshold
            locations = np.where(result >= threshold)

            # If any match is found above the threshold, consider it as a car
            if len(locations[0]) > 0:
                correct_num += 1
    # return the correct number of predict
    return correct_num

In [11]:
# This car_models array and input shape need to define if the model is only load from the previous pretrained model.
# No need to initialize if the model is trained before running this code
car_models = ['HATCHBACK', 'SEDAN', 'SUV']
input_shape = (128, 128, 1)

# This function is to predict the image of the car is in what category
def classify_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # If the image is available
    if image is not None:
        # Resize the image to match the input shape of the model
        image = cv2.resize(image, input_shape[:2])
        # Convert the image from RGB to Grayscale
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        # Normalize the pixel values
        image = image.astype('float32') / 255.0
        # Reshape the image to match the input shape of the model
        image = np.reshape(image, (1,) + input_shape)
        # Make prediction using the train model above to using the pretrained model
        predictions = model.predict(image)
        # Get the predicted class index
        predicted_class_index = np.argmax(predictions[0])
        # Get the corresponding car model labels
        car_model_labels = [car_models[i] for i in range(len(car_models))]
        # Get the percentages of matching of input image car with the predicted car
        percentages = predictions[0] * 100
        # If the precentage is larger than 50, then predict the car model and display the result in the console
        if percentages[predicted_class_index] >= 50:
            print("Matched: {} ({}% confidence)".format(car_model_labels[predicted_class_index], percentages[predicted_class_index]))
            return percentages[0], percentages[1], percentages[2]
        # Else the car is not match to any model
        else:
            print("Not matched")

        # Display the distribution of similarities
        for i in range(len(car_models)):
            print("{}: {}%".format(car_model_labels[i], percentages[i]))
    # If the image is not available, print invalid image
    else:
        print("Invalid image")

In [12]:
# This function is use to predict whether the four image can conclude what type of the car is
def predict(image_path_arr):
    # Set up the array sum_car to zero at the beginning, it is use to put the sum of percentage each category of the car  
    sum_car = [0, 0, 0]
    # Define the car category
    car_category = ['hatchback', 'sedan', 'suv']
    # Loop all the image 
    for i in range(len(image_path_arr)):
        # Set the variable hatchback, sedan and suv as zero 
        hatchback = 0
        sedan = 0
        suv = 0
        image_path = image_path_arr[i]
        # Classify the input image car is which category
        hatchback, sedan, suv = classify_image(image_path)
        # Add the percentage into the sum_car variable
        sum_car[0] += hatchback
        sum_car[1] += sedan
        sum_car[2] += suv
        print(f'hatchback: {hatchback}, sedan: {sedan}, suv: {suv}')            
    
    # Find the max percentage of the three car category
    max_percentage = sum_car[0]
    car_index = 0
    for x in range(2):
        if sum_car[x+1] > max_percentage:
            max_percentage = sum_car[x+1]
            car_index = x+1
    
    # If the largest percentage divide by 4 is larger than 50, then predict which car category and show the percentage of confidence
    if max_percentage/4 > 50:
        print(f'Predicted car : {car_category[car_index]}, Predicted Percentage : {max_percentage/4}')
    # Else the image had to be retake in order to predict the car more accurate
    else:
        print("The image is not enough to prove the car model.")

In [13]:
# This function is use to predict the car using the four input image
# It is first predict whether the image had car. If the image contain car then only it will predict which category the car is in
def predict_car_category(image_path):
    # Define the sum_num to zero, it is use to see the four input image had match to how many template image that contain car
    sum_num = 0
    # Loop all the input image
    for i in range(len(image_path)):
        # Compute the number of the image that detected car
        num = image_detect(image_path[i], i+1)
        sum_num += num
    
    # If the total of detected image is larger than 5, then only the model will predict the car category
    if sum_num > 5:
        predict(['ans1.png',
                 'ans2.png',
                 'ans3.png',
                 'ans4.png'])
    # Or else the image had to be retake in order to predict the car category
    else:
        print("Input images were incompatible with car classification model")

In [22]:
# This is to save the model after the model had been train in order to use it to predict the car category in future
model.save('model128_64_128_128_128.h5')

In [23]:
# Load the saved pretrained model
model = load_model('model128_64_128_128_128.h5')

In [24]:
# This is an example of predicting car category
predict_car_category(['Testing Data/AXIA LEFT SIDE.jpg',
                      'Testing Data/AXIA FRONT.jpg',
                      'Testing Data/AXIA BACK.jpg',
                      'Testing Data/AXIA RIGHT SIDE.jpg'])

1/1 [==============================] - 0s 53ms/step
Matched: HATCHBACK (99.4267578125% confidence)
hatchback: 99.4267578125, sedan: 0.004365134984254837, suv: 0.5688737630844116
1/1 [==============================] - 0s 20ms/step
Matched: SEDAN (83.45654296875% confidence)
hatchback: 0.007198809180408716, sedan: 83.45654296875, suv: 16.5362548828125
1/1 [==============================] - 0s 20ms/step
Matched: HATCHBACK (97.55409240722656% confidence)
hatchback: 97.55409240722656, sedan: 2.305206775665283, suv: 0.14070414006710052
1/1 [==============================] - 0s 21ms/step
Matched: HATCHBACK (98.9119644165039% confidence)
hatchback: 98.9119644165039, sedan: 0.0001395588624291122, suv: 1.0878931283950806
Predicted car : hatchback, Predicted Percentage : 73.97500336135272


In [17]:
# This is an example of predicting car category
predict_car_category(['Testing Data/CRV LEFT SIDE.jpg',
                      'Testing Data/CRV FRONT.jpg',
                      'Testing Data/CRV BACK.jpg',
                      'Testing Data/CRV RIGHT SIDE.jpg'])

1/1 [==============================] - 0s 21ms/step
Matched: SUV (99.9999771118164% confidence)
hatchback: 2.822132591973059e-05, sedan: 1.3013168320696877e-08, suv: 99.9999771118164
1/1 [==============================] - 0s 19ms/step
Matched: SEDAN (96.63973999023438% confidence)
hatchback: 0.47029909491539, sedan: 96.63973999023438, suv: 2.8899638652801514
1/1 [==============================] - 0s 19ms/step
Matched: SEDAN (99.89816284179688% confidence)
hatchback: 0.0007564264233224094, sedan: 99.89816284179688, suv: 0.10106825828552246
1/1 [==============================] - 0s 18ms/step
Matched: SUV (99.99995422363281% confidence)
hatchback: 9.922285926222685e-07, sedan: 4.157980583840981e-05, suv: 99.99995422363281
Predicted car : suv, Predicted Percentage : 50.74774086475372


In [18]:
# This is an example of predicting car category
predict_car_category(['Testing Data/GRAND LEFT SIDE.jpg',
                      'Testing Data/GRAND FRONT.jpg',
                      'Testing Data/GRAND BACK.jpg',
                      'Testing Data/GRAND RIGHT SIDE.jpg'])

1/1 [==============================] - 0s 21ms/step
Matched: SUV (99.47535705566406% confidence)
hatchback: 0.009767252951860428, sedan: 0.5148714184761047, suv: 99.47535705566406
1/1 [==============================] - 0s 19ms/step
Matched: HATCHBACK (96.7442855834961% confidence)
hatchback: 96.7442855834961, sedan: 1.3586804866790771, suv: 1.8970277309417725
1/1 [==============================] - 0s 19ms/step
Matched: HATCHBACK (99.62906646728516% confidence)
hatchback: 99.62906646728516, sedan: 0.26631852984428406, suv: 0.10461023449897766
1/1 [==============================] - 0s 19ms/step
Matched: HATCHBACK (55.693416595458984% confidence)
hatchback: 55.693416595458984, sedan: 44.28823471069336, suv: 0.0183410681784153
Predicted car : hatchback, Predicted Percentage : 63.019133974798024


In [19]:
# This is an example of predicting car category
predict_car_category(['Testing Data/KELISA LEFT SIDE.jpg',
                      'Testing Data/KELISA FRONT.jpg',
                      'Testing Data/KELISA BACK.jpg',
                      'Testing Data/KELISA RIGHT SIDE.jpg'])

1/1 [==============================] - 0s 29ms/step
Matched: HATCHBACK (64.30679321289062% confidence)
hatchback: 64.30679321289062, sedan: 35.52077865600586, suv: 0.17242205142974854
1/1 [==============================] - 0s 20ms/step
Matched: SEDAN (99.99774932861328% confidence)
hatchback: 0.0022515878081321716, sedan: 99.99774932861328, suv: 7.442391165568552e-07
1/1 [==============================] - 0s 22ms/step
Matched: SUV (80.10796356201172% confidence)
hatchback: 0.18735335767269135, sedan: 19.70467758178711, suv: 80.10796356201172
1/1 [==============================] - 0s 20ms/step
Matched: SUV (61.9642448425293% confidence)
hatchback: 9.072932243347168, sedan: 28.962820053100586, suv: 61.9642448425293
The image is not enough to prove the car model.


In [20]:
# This is an example of predicting car category
predict_car_category(['Testing Data/VIOS LEFT SIDE.jpg',
                      'Testing Data/VIOS FRONT.jpg',
                      'Testing Data/VIOS BACK.jpg',
                      'Testing Data/VIOS RIGHT SIDE.jpg'])

1/1 [==============================] - 0s 32ms/step
Matched: SEDAN (93.58775329589844% confidence)
hatchback: 6.410701751708984, sedan: 93.58775329589844, suv: 0.0015553187113255262
1/1 [==============================] - 0s 19ms/step
Matched: SUV (98.82181549072266% confidence)
hatchback: 1.1781058311462402, sedan: 7.992513565113768e-05, suv: 98.82181549072266
1/1 [==============================] - 0s 20ms/step
Matched: SEDAN (85.68953704833984% confidence)
hatchback: 0.000276043894700706, sedan: 85.68953704833984, suv: 14.310185432434082
1/1 [==============================] - 0s 19ms/step
Matched: SEDAN (99.83934783935547% confidence)
hatchback: 0.16063249111175537, sedan: 99.83934783935547, suv: 1.0119579201273154e-05
Predicted car : sedan, Predicted Percentage : 69.77917952718235


In [21]:
# This is an example of predicting car category
predict_car_category(['Testing Data/CAT SIDE.jpg',
                      'Testing Data/CAT FRONT.jpg',
                      'Testing Data/CAT BACK.jpg',
                      'Testing Data/CAT SIDE2.jpg'])

Input images were incompatible with car classification model
